In [19]:
import requests
import pandas as pd
import json

def search_recording(artist_name, release_title):
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    #print(data)
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][1]['id']
    return None

# Example usage:
artist = "Chico Hamilton"
album = "Peregrinations"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

Recording MBID for 'Peregrinations' by 'Chico Hamilton': 62cf2ca2-087f-4701-892e-de974e91ae3c


In [20]:
def get_credits_for_recording(release_mbid):
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels + release-groups",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
    #print(data)

    credits = []
    
    if data and data.get('artist-credit'):
       for track in data['media'][0]['tracks']:
            for credit in track['recording']['relations']:
                if credit['attributes'] == []:
                    attributes = 'Not Found'
                else:
                    attributes = credit['attributes']
                credits.append((track['number'], track['title'], credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))
            for credit in track['artist-credit']:
                credits.append((track['number'], track['title'], credit.get('name', 'Not Found'), credit.get('attributes', 'Not Found'), credit.get('artist', {}).get('id', 'Not Found')))
    if data['relations']:
       for credit in data['relations']:
           trackNum = 'Not Found'
           trackTitle = 'Not Found'
           if credit['attributes'] == [] or not credit['attributes']:
               attributes = 'Not Found'
           else:
               attributes = credit['attributes']
           credits.append((trackNum, trackTitle, credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))
    #print(credits)

    #dataTest = data['media'][0]['tracks'][0]['recording']['relations'][2]['artist']['name']
    #print(dataTest)

    #credits = [{"Artist" : credit.get('artist', {}).get('name', 'Not Found'), "Role" : credit['attributes'], "Track Number" : track['number'], "Track Title" : track['title']} for track in data['media'][0]['tracks'] for credit in track['recording']['relations']]

    creditNames = []
    creditList = []
    for credit in credits:
        if credit[2] not in creditNames:
            creditNames.append(credit[2])
            creditList.append((credit[2],credit[3], credit[4]))
    print(creditNames)
    print("\n")
    print(creditList)
    
    return credits

In [21]:
#Example usage:
if release_mbid:
    albumCredits = get_credits_for_recording(release_mbid)
    #print(albumCredits)
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

# Create the DataFrame
#df = pd.DataFrame(albumCredits, columns = ['Track Number', 'Track Title', 'Artist', 'Role', 'Artist_MBID'])

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)


# Display the DataFrame
#df

['Baker Bigsby', 'Keg Johnson', 'Chico Hamilton', 'Bob Cato', 'Don Hahn', 'Steve Maslow', 'Ahkmed Abdullah', 'Joe Beck', 'Arthur Blythe', 'Barry Finnerty', 'Arnie Lawrence', 'Jerry Peters', 'Steve Turre', 'Stanley Crouch', 'George Butler', 'Charlotte Politte', 'Julia Waters', 'Luther Waters', 'Maxine Waters', 'Oren Waters']


[('Baker Bigsby', 'Not Found', '7cf86f47-fbb0-44a5-949e-ba539b0ef93c'), ('Keg Johnson', 'Not Found', '6c049421-822f-4367-8f68-8456262dd972'), ('Chico Hamilton', 'Not Found', '699067a0-ae8c-45fc-82c1-6227d91f9bb4'), ('Bob Cato', 'Not Found', 'fc21ed6e-4d7c-45b0-9b66-5ff158d7636c'), ('Don Hahn', 'Not Found', '3169f754-035e-4bbd-8ee4-35488fd6fb5d'), ('Steve Maslow', 'Not Found', '50fd20f6-b97f-4827-b001-f79ae2cb207b'), ('Ahkmed Abdullah', ['congas'], 'e04f56ea-ad76-46e6-8e37-f96539fa1519'), ('Joe Beck', ['guitar'], 'e8e98ca1-9a4f-4148-90c7-8983da9ed34a'), ('Arthur Blythe', ['horn'], '91652f18-ce2d-40e1-b6ab-07179a82fb12'), ('Barry Finnerty', ['guitar'], '9f3c93ae-f1b